向量化环境（Vectorized Environments）是一种将多个独立环境堆叠到一个单一环境中的方法，这使得在每一步中，而不是在一个环境上训练RL代理，我们可以在n个环境上训练它。这样，传递给环境的动作现在是一个向量（维度为n），观察（observations）、奖励（rewards）和结束信号（dones）也是如此。对于非数组观察空间（如Dict或Tuple），其中不同子空间可能有不同的形状，子观察是向量（维度n）。

### 使用场景

- **多环境训练**：当你需要同时在多个环境上训练RL代理以加快学习过程或提高泛化能力时。
- **高效利用硬件**：通过并行执行多个环境，可以更高效地利用CPU或GPU资源。

### 参数说明

- **Box, Discrete, Dict, Tuple**：这些是OpenAI Gym环境中观察空间和动作空间的类型。向量化环境支持这些类型。
- **Multi Processing**：是否支持多进程。`DummyVecEnv`不支持多进程，而`SubprocVecEnv`支持。

### 实现功能

- **自动环境重置**：向量化环境在每个episode结束时自动重置环境。
- **支持不同观察空间类型**：无论是基本类型（如Box, Discrete）还是复合类型（如Dict, Tuple），向量化环境都能处理。
- **多进程支持**：通过`SubprocVecEnv`，可以在不同的进程中并行运行多个环境，从而提高训练速度。

### 注意事项

- 使用框架堆叠（frame-stacking）或标准化（normalization）时，需要向量化环境。
- 向量化环境在`done[i]`为真时自动重置，返回的观察是下一个episode的第一个观察，而不是刚结束的episode的最后一个观察。可以通过`info`字典中的`terminal_observation`键访问真正的最终观察。
- 自定义`VecEnv`时应提供`info`字典中的`terminal_observation`键。
- 在使用`SubprocVecEnv`且采用`forkserver`或`spawn`启动方法（Windows默认）时，必须将代码包装在`if __name__ == "__main__":`中。Linux默认的启动方法是`fork`，它不是线程安全的，可能会导致死锁。

### 示例代码

接下来，让我们通过一个示例代码来展示如何使用`DummyVecEnv`和`SubprocVecEnv`来创建向量化环境，并在其中运行一个简单的环境。这段代码将会创建多个环境实例，并展示如何在这些环境上同步执行动作和处理观察。

请注意，下面的代码需要你已经安装了`stable_baselines3`和`gym`库。

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3 import PPO

# 创建环境的方法
def make_env(env_id, rank, seed=0):
    def _init():
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env
    return _init

env_id = "CartPole-v1"
num_envs = 4

# 使用DummyVecEnv创建单进程向量化环境
envs = [make_env(env_id, i) for i in range(num_envs)]
vec_env = DummyVecEnv

(envs)

# 使用SubprocVecEnv创建多进程向量化环境
# vec_env = SubprocVecEnv(envs)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25000)
```

这段代码首先定义了一个创建环境的方法`make_env`，然后使用`DummyVecEnv`或`SubprocVecEnv`来创建一个向量化环境，其中包含多个`CartPole-v1`环境实例。接着，使用PPO算法的模型来训练这个向量化环境。


## VecEnv API vs Gym API


### SB3 VecEnv API 与 Gym API 的不同

1. **`reset()` 方法**:
   - 在SB3中，`reset()`方法仅返回观察值（`obs = vec_env.reset()`），而不是一个元组。`reset`时的信息被存储在`vec_env.reset_infos`中。
   - 在Gym中，`reset()`可能返回一个元组，包括观察值和额外的信息。

2. **环境自动重置**:
   - SB3中，只需最初调用一次`vec_env.reset()`，之后环境会自动重置，且`reset_infos`会自动更新。
   - 这与Gym API不同，在Gym中，每个episode结束后通常需要手动调用`reset()`。

3. **`step()` 方法**:
   - SB3的`step(actions)`方法接受一个数组作为输入（批量大小对应环境数），并返回一个4元组（`obs, rewards, dones, infos`），而非Gym 0.26+的5元组。
   - `dones`在SB3中等价于Gym中的`terminated`或`truncated`。

4. **结束一个episode**:
   - SB3提供了`infos[env_idx]["TimeLimit.truncated"]`来指示一个episode是否因为超时/截断而结束，与Gym 0.26+中的`terminated`和`truncated`互斥。
   - 在episode结束时，因为环境自动重置，SB3还提供了`terminal_observation`键，包含最后一个观察值，这对于在截断时的引导（bootstrapping）很有用。

5. **渲染限制**:
   - 为了克服Gym只允许每个环境实例一个渲染模式的限制，SB3推荐使用`render_mode="rgb_array"`，这样既可以获取图像的numpy数组，也可以用OpenCV显示。

6. **`reset()` 方法不接受参数**:
   - 如果想要设置伪随机生成器的种子或传递选项，应该在调用`reset()`之前调用`vec_env.seed(seed=seed)`/`vec_env.set_options(options)`。

7. **访问底层Gym环境的方法和属性**:
   - 可以通过`vec_env.get_attr("attribute_name")`、`vec_env.env_method("method_name", args1, args2, kwargs1=kwargs1)`和`vec_env.set_attr("attribute_name", new_value)`访问、调用和设置底层Gym环境的方法和属性。

### 示例代码

为了提供一个如何在实际中应用这些差异的例子，假设我们正在使用VecEnv来训练一个RL模型。以下代码示例展示了如何初始化环境、进行重置、执行动作、处理返回值，以及如何渲染环境状态。

由于代码运行需要具体的环境和框架设置，这里仅提供一个概念性的框架：

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv

# 初始化环境
env_id = "CartPole-v1"
envs = [lambda: gym.make(env_id) for _ in range(4)]  # 创建4个环境的列表
vec_env = DummyVecEnv(envs)  # 使用DummyVecEnv来包装这些环境

# 重置环境
obs = vec_env.reset()

# 执行动作
actions = [env.action_space.sample() for env in vec_env.envs]  # 随机选择动作
obs, rewards, dones, infos = vec_env.step(actions)

# 处理返回值
print("Observations:", obs)
print("Rewards:", rewards

)
print("Dones:", dones)
print("Infos:", infos)

# 渲染环境状态
# 注意：实际使用时，需要根据环境支持的渲染模式来调用render方法
```

请注意，这段代码是一个概念性示例，具体实现时需要根据你的环境和需求进行调整。在实际应用中，你可能需要安装并导入相应的库（如Stable-Baselines3和Gym），并且确保你的环境支持所需的操作。

## 修改矢量化环境属性 Modifying Vectorized Environments Attributes

如果您计划在使用环境时修改其属性（例如，修改指定在进行多任务学习时为部分训练执行的任务的属性，或环境动态的参数），则必须公开设置器方法。事实上，直接访问回调中的环境属性可能会导致意外行为，因为环境可以被包装（使用gym或VecEnv包装器，Monitor包装器就是一个例子）。

考虑以下自定义环境的示例：

In [ ]:
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3.common.env_util import make_vec_env


class MyMultiTaskEnv(gym.Env):

  def __init__(self):
      super().__init__()
      """
      A state and action space for robotic locomotion.
      The multi-task twist is that the policy would need to adapt to different terrains, each with its own
      friction coefficient, mu.
      The friction coefficient is the only parameter that changes between tasks.
      mu is a scalar between 0 and 1, and during training a callback is used to update mu.
      """
      ...

  def step(self, action):
    # Do something, depending on the action and current value of mu the next state is computed
    return self._get_obs(), reward, done, truncated, info

  def set_mu(self, new_mu: float) -> None:
      # Note: this value should be used only at the next reset
      self.mu = new_mu

# Example of wrapped env
# env is of type <TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>
env = gym.make("CartPole-v1")
# To access the base env, without wrapper, you should use `.unwrapped`
# or env.get_wrapper_attr("gravity") to include wrappers
env.unwrapped.gravity
# SB3 uses VecEnv for training, where `env.unwrapped.x = new_value` cannot be used to set an attribute
# therefore, you should expose a setter like `set_mu` to properly set an attribute
vec_env = make_vec_env(MyMultiTaskEnv)
# Print current mu value
# Note: you should use vec_env.env_method("get_wrapper_attr", "mu") in Gymnasium v1.0
print(vec_env.env_method("get_wrapper_attr", "mu"))
# Change `mu` attribute via the setter
vec_env.env_method("set_mu", "mu", 0.1)

在此示例中，env.mu 无法直接访问/更改，因为它被包装在 VecEnv 中，并且可以用其他包装器包装（有关详细说明，请参阅 GH#1573）。相反，回调应通过矢量化环境的 env_method 方法使用 set_mu 方法。

In [ ]:
from itertools import cycle

class ChangeMuCallback(BaseCallback):
  """
  This callback changes the value of mu during training looping
  through a list of values until training is aborted.
  The environment is implemented so that the impact of changing
  the value of mu mid-episode is visible only after the episode is over
  and the reset method has been called.
  """"
  def __init__(self):
    super().__init__()
    # An iterator that contains the different of the friction coefficient
    self.mus = cycle([0.1, 0.2, 0.5, 0.13, 0.9])

  def _on_step(self):
    # Note: in practice, you should not change this value at every step
    # but rather depending on some events/metrics like agent performance/episode termination
    # both accessible via the `self.logger` or `self.locals` variables
    self.training_env.env_method("set_mu", next(self.mus))
    
# 然后，此回调可用于在训练期间安全地修改环境属性，因为它调用环境 setter 方法。

## 代码解读：

让我们逐行分析并丰富上述代码，以便更好地理解如何在训练过程中安全修改向量化环境属性。

### 自定义环境类：MyMultiTaskEnv

```python
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_util import make_vec_env

class MyMultiTaskEnv(gym.Env):
    def __init__(self):
        super().__init__()
        # 定义状态空间和动作空间，这里省略了具体定义
        ...
```
这部分代码定义了一个自定义环境`MyMultiTaskEnv`，它继承自`gym.Env`。这个环境旨在模拟不同地形下的机器人运动，其中`mu`表示不同地形的摩擦系数，是需要在训练过程中修改的属性。

```python
    def step(self, action):
        # 根据动作和当前mu值计算下一个状态
        return self._get_obs(), reward, done, truncated, info
```
`step`方法是环境的核心，根据当前动作和摩擦系数`mu`的值来计算下一个状态和奖励。

```python
    def set_mu(self, new_mu: float) -> None:
        # 设置新的mu值，注意这个值只会在下次reset后生效
        self.mu = new_mu
```
`set_mu`方法允许外部代码安全地修改`mu`的值。这个设计是为了避免直接修改属性可能导致的问题，特别是在多层包装的环境中。

### 环境实例化和修改属性

```python
# 实例化一个CartPole环境作为常规Gym环境的示例
env = gym.make("CartPole-v1")
# 直接访问基础环境的属性，跳过所有包装器
env.unwrapped.gravity
```
这部分代码展示了如何在不使用`VecEnv`的情况下实例化一个环境，并直接访问其属性。这对于了解包装器如何影响属性访问很有帮助。

```python
# 使用make_vec_env创建向量化环境的实例
vec_env = make_vec_env(MyMultiTaskEnv)
```
这里使用`make_vec_env`函数创建了一个向量化环境实例。这是SB3中创建并行环境以加速训练的推荐方法。

### 修改向量化环境属性

```python
# 使用env_method修改mu值
vec_env.env_method("set_mu", 0.1)
```
通过`env_method`调用自定义环境的`set_mu`方法来修改`mu`的值。这种方法确保了即使在环境被包装的情况下也能安全修改属性。

### 回调函数：ChangeMuCallback

```python
from itertools import cycle
from stable_baselines3.common.callbacks import BaseCallback

class ChangeMuCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        # 初始化一个循环迭代器，包含不同的摩擦系数值
        self.mus = cycle([0.1, 0.2, 0.5, 0.13, 0.9])

    def _on_step(self):
        # 在训练的每一步中调用，根据需要修改mu值
        self.training_env.env_method("set_mu", next(self.mus))
```
`ChangeMuCallback`是一个自定义回调，它在训练过程中循环地修改`mu`值。通过在`_on_step`方法中调用`env_method`，它安全地修改了环境属性，避免了直接访问可能导致的问题。

这个回调可以在创建模型时作为参数传入，允许在训练过程中根据特定事件或性能指标动态调整环境属性。

通过以上详细解释和代码丰

富，我们可以看到如何在SB3中安全且有效地修改向量化环境的属性，特别是在进行多任务学习或需要根据训练进度调整环境参数时。这种方法增加了训练过程的灵活性，允许更精细地控制环境的行为。

上述内容详细讲解了在使用Stable-Baselines3（SB3）进行强化学习训练时，如何安全地修改向量化环境（VecEnv）中环境的属性。这在多任务学习或需要在训练过程中调整环境动力学参数的场景中尤其有用。关键点包括：

### 修改环境属性的重要性

- 在多任务学习中，可能需要在训练过程中修改环境属性（例如，不同任务的摩擦系数）。
- 直接访问和修改环境属性可能导致意外行为，因为环境可能会被多层包装（例如使用gym或VecEnv的包装器）。

### 自定义环境示例

- 创建了一个自定义环境`MyMultiTaskEnv`，其中包含了一个用于机器人运动的状态空间和动作空间，以及模拟不同地形的摩擦系数`mu`。
- `mu`的值在训练过程中通过回调函数更新，而在环境的每个新episode开始时才实际使用新值。

### 修改属性的正确方式

- 为了在环境中正确设置属性，应当暴露一个setter方法（例如，`set_mu`），而不是直接设置属性值。
- 在使用VecEnv时，不能直接通过`env.unwrapped.x = new_value`这样的方式访问或修改属性，因为它可能不适用于向量化环境或被其他包装器包装的环境。
- 应使用`vec_env.env_method("method_name", args)`来调用环境的方法，如示例中通过`vec_env.env_method("set_mu", 0.1)`来修改`mu`值。

### 使用回调函数安全修改属性

- `ChangeMuCallback`是一个回调函数示例，它在训练过程中通过循环遍历一组`mu`值来修改环境的摩擦系数。
- 该回调利用`env_method`安全地调用了设置`mu`的方法，从而在不直接访问环境属性的情况下修改了它们。
- 在实践中，修改这些属性的操作应根据一些事件/指标（如代理性能或episode结束）而非每一步都进行。

### 示例代码中的关键点

- 示例中展示了如何定义自定义环境、如何使用`env_method`来安全调用环境中的setter方法，以及如何在训练过程中通过回调函数修改环境属性。
- 这种方法确保了即使环境被多个包装器包装，也能安全、有效地修改环境属性，是在进行复杂训练策略时的重要技巧。

这种在训练过程中修改环境属性的能力是进行多任务学习和适应动态环境变化的关键，能够显著提高模型的灵活性和适应性。通过正确使用setter方法和回调函数，可以确保这些修改既安全又有效，避免了直接访问和修改可能引起的问题。

## 矢量化环境包装器Vectorized Environments Wrappers

在强化学习中，环境包装器（wrappers）是一种强大的工具，允许你在不完全重新定义环境的情况下修改或增强环境。对于向量化环境（VecEnv），`VecEnvWrapper`提供了一种在多个环境上同时操作的方式，类似于Gym中的`gym.Wrapper`。

### 示例解释：VecExtractDictObs

以下示例展示了如何创建一个`VecEnvWrapper`来过滤字典观测中的特定键值。这对于处理返回复合观察（例如，包含多个键的字典）的环境特别有用。

```python
import numpy as np
from stable_baselines3.common.vec_env.base_vec_env import VecEnv, VecEnvStepReturn, VecEnvWrapper

class VecExtractDictObs(VecEnvWrapper):
    """
    A vectorized wrapper for filtering a specific key from dictionary observations.
    Similar to Gym's FilterObservation wrapper.
    
    :param venv: The vectorized environment
    :param key: The key of the dictionary observation
    """
    def __init__(self, venv: VecEnv, key: str):
        self.key = key
        super().__init__(venv=venv, observation_space=venv.observation_space.spaces[self.key])
```

这个`VecExtractDictObs`类构造函数接受一个向量化环境（`venv`）和一个键（`key`），用于指定要从观察中提取的字典键。在初始化时，它调用父类构造函数，并设置新的观察空间为原始环境观察空间中对应键的空间。

```python
    def reset(self) -> np.ndarray:
        obs = self.venv.reset()
        return obs[self.key]
```
`reset`方法重置环境，并返回指定键的观察值。这意味着当环境被重置时，只有该键对应的观察部分被返回。

```python
    def step_async(self, actions: np.ndarray) -> None:
        self.venv.step_async(actions)
```
`step_async`方法是向量化环境中异步执行动作的方法。这里，它简单地将动作传递给包装的环境。

```python
    def step_wait(self) -> VecEnvStepReturn:
        obs, reward, done, info = self.venv.step_wait()
        return obs[self.key], reward, done, info
```
`step_wait`方法等待动作执行完成，并返回过滤后的观察值（只包含指定的键）、奖励、结束标志和额外信息。

### 使用示例

```python
env = DummyVecEnv([lambda: gym.make("FetchReach-v1")])
# Wrap the VecEnv
env = VecExtractDictObs(env, key="observation")
```
这部分代码首先使用`DummyVecEnv`创建了一个向量化环境实例，然后通过`VecExtractDictObs`包装器对其进行包装，以便只从观察中提取键为`"observation"`的部分。

通过这种方式，`VecEnvWrapper`提供了一种灵活的方法来修改和增强向量化环境的行为，而无需改动环境本身的代码。这种方法对于实验和快速原型开发尤其有用，因为它允许研究者和开发者在不改变底层环境实现的情况下，轻松地测试不同的观察处理策略和环境修改。

# VecEnv

`VecEnv`是Stable-Baselines3中用于处理向量化环境的一个抽象类。向量化环境允许同时运行多个环境实例，这对于加速训练过程非常有用。以下是`VecEnv`类及其方法的详细解读：

### 类定义

```python
class stable_baselines3.common.vec_env.VecEnv(num_envs, observation_space, action_space)
```

#### 参数

- `num_envs (int)`: 环境的数量，即你希望并行运行的环境实例数。
- `observation_space (Space)`: 观察空间，定义了环境观察的形状和数据类型。
- `action_space (Space)`: 动作空间，定义了可能的动作的形状和数据类型。

### 方法

#### `abstractclose()`

- 清理环境的资源。
- **返回类型**: None

#### `abstractenv_is_wrapped(wrapper_class, indices=None)`

- 检查环境是否被给定的包装器包装。
- **参数**:
  - `wrapper_class (Type[Wrapper])`: 要检查的包装器类。
  - `indices (None | int | Iterable[int])`: 要检查的环境索引。
- **返回**:
  - **返回类型**: List[bool]，每个环境是否被包装的布尔值列表。

#### `abstractenv_method(method_name, *method_args, indices=None, **method_kwargs)`

- 调用向量化环境实例方法。
- **参数**:
  - `method_name (str)`: 要调用的环境方法名称。
  - `indices (None | int | Iterable[int])`: 要调用方法的环境索引。
  - `method_args`和`method_kwargs`: 方法调用时传递的位置参数和关键字参数。
- **返回**:
  - **返回类型**: List[Any]，环境方法调用的返回值列表。

#### `abstractget_attr(attr_name, indices=None)`

- 从向量化环境返回属性。
- **参数**:
  - `attr_name (str)`: 要返回值的属性名称。
  - `indices (None | int | Iterable[int])`: 要获取属性的环境索引。
- **返回**:
  - **返回类型**: List[Any]，所有环境中`attr_name`的值的列表。

#### `get_images()`

- 返回每个环境的RGB图像（如果可用）。
- **返回类型**: Sequence[ndarray | None]

#### `render(mode=None)`

- 渲染Gym环境。
- **参数**:
  - `mode (str | None)`: 渲染类型。
- **返回类型**: ndarray | None

#### `abstractreset()`

- 重置所有环境，并返回观察数组，或观察数组的元组。
- **返回**:
  - **返回类型**: ndarray | Dict[str, ndarray] | Tuple[ndarray, …]，观察值。

#### `seed(seed=None)`

- 为所有环境设置随机种子。
- **参数**:
  - `seed (int | None)`: 随机种子。None表示完全随机种子。
- **返回**:
  - **返回类型**: Sequence[None | int]，每个环境的种子列表。

#### `abstractset_attr(attr_name, value, indices=None)`

- 在向量化环境内设置属性。
- **参数**:
  - `attr_name (str)`: 要赋新值的属性名称。
  - `value (Any)`: 要赋给`attr_name`的值。
  - `indices (None | int | Iterable[int])`: 要赋值的环境索引。
- **返回类型**: None

#### `set_options(options=None)`

- 为所有环境设置环境选项。
- **参数**:
  - `options (List[Dict] | Dict | None)`: 下一次重置时传递给每个环境的环境选项字典或字典列表。
- **返回类型**: None

#### `step(actions)`

- 使用给定动作步进

环境。
- **参数**:
  - `actions (ndarray)`: 动作。
- **返回**:
  - **返回类型**: Tuple[ndarray | Dict[str, ndarray] | Tuple[ndarray, …], ndarray, ndarray, List[Dict]]，观察值、奖励、完成标志和信息。

#### `abstractstep_async(actions)`

- 告诉所有环境开始执行给定动作的步骤。使用`step_wait()`获取步骤的结果。
- **参数**:
  - `actions (ndarray)`: 动作。
- **返回类型**: None

#### `abstractstep_wait()`

- 等待`step_async()`执行的步骤完成。
- **返回**:
  - **返回类型**: Tuple[ndarray | Dict[str, ndarray] | Tuple[ndarray, …], ndarray, ndarray, List[Dict]]，观察值、奖励、完成标志和信息。

这个`VecEnv`类及其方法为在并行环境中进行强化学习训练提供了强大的基础设施。通过允许同时处理多个环境实例，它显著提高了训练效率和效果。

为了演示如何使用`VecEnv`类及其方法，我们将通过一个使用`DummyVecEnv`的示例，来展示如何创建向量化环境、执行动作、重置环境以及获取观察结果。`DummyVecEnv`是`VecEnv`的一个实现，用于在单个进程中同步运行多个环境实例。这对于调试和开发时测试代码非常有用。

### 示例代码

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np

# 定义环境创建函数，这个函数会创建并返回一个Gym环境实例
def make_env(env_name):
    def _init():
        return gym.make(env_name)
    return _init

env_name = "CartPole-v1"  # 使用CartPole环境作为示例
num_envs = 4  # 并行运行的环境数量

# 创建向量化环境
envs = [make_env(env_name) for _ in range(num_envs)]
vec_env = DummyVecEnv(envs)

# 重置环境，开始新的episode
observations = vec_env.reset()

# 执行一些动作
for _ in range(10):  # 演示10个时间步
    actions = np.array([env.action_space.sample() for env in vec_env.envs])
    obs, rewards, dones, infos = vec_env.step(actions)
    print(f"Observations: {obs}")
    print(f"Rewards: {rewards}")
    print(f"Dones: {dones}")
    # 如果任何环境完成了，重置该环境
    for i, done in enumerate(dones):
        if done:
            vec_env.env_method('reset', indices=[i])

# 关闭环境，清理资源
vec_env.close()
```

### 代码解释

1. **环境创建函数**(`make_env`): 这个函数接受环境名称并返回一个初始化函数，该函数在被调用时创建并返回一个Gym环境实例。这样的设计允许`DummyVecEnv`在内部初始化多个环境实例。

2. **创建向量化环境**:
   - 首先，创建一个环境名称列表，每个元素都是通过`make_env`函数返回的初始化函数。
   - 然后，使用`DummyVecEnv`将这些环境包装成一个向量化环境，允许我们并行运行和管理多个环境实例。

3. **重置环境**:
   - 通过调用`vec_env.reset()`来重置所有环境，开始新的episode。这将返回初始观察结果数组。

4. **执行动作和环境交互**:
   - 在一个循环中，我们随机生成动作（通过调用每个环境的`action_space.sample()`），然后调用`vec_env.step(actions)`来在每个环境中执行这些动作。
   - `step`方法返回四个值：`obs`（新的观察结果），`rewards`（奖励），`dones`（是否结束），和`infos`（额外的信息）。

5. **检查环境状态**:
   - 循环中，检查每个环境是否达到了结束状态（`done`）。如果是，使用`vec_env.env_method('reset', indices=[i])`来只重置完成的环境。

6. **关闭环境**:
   - 使用`vec_env.close()`来关闭所有环境并清理资源。这是在完成所有交互后的重要步骤。

这个示例演示了如何使用`DummyVecEnv`来创建和管理多个Gym环境的向量化版本。这种方式使得并行环境管理变得简单，同时也提高了数据采集和训练过程的效率。通过这种方式，可以在单个进程内同时运行和监控多个环境实例，是进行强化学习研究和实验的一个非常有用的工具。

## DummyVecEnv

`DummyVecEnv` 是 Stable Baselines3 库中用于向量化多个环境的一个类，它在当前Python进程中顺序调用每个环境。这对于计算上简单的环境（如`CartPole-v1`）特别有用，因为在这种情况下，多进程或多线程的开销可能会超过环境计算时间。`DummyVecEnv`也可以用于那些需要向量化环境但只想用单个环境训练的强化学习方法。

### 参数

- `env_fns (List[Callable[[], Env]])`：一个返回环境实例的函数列表。每个函数在被调用时都应该创建并返回一个新的环境实例。

### 方法

#### `close()`

- 清理环境的资源。
- **返回类型**：None

#### `env_is_wrapped(wrapper_class, indices=None)`

- 检查工作环境是否被给定的包装器包装。
- **参数**：
  - `wrapper_class (Type[Wrapper])`：包装器类。
  - `indices (None | int | Iterable[int])`：要检查的环境索引。
- **返回类型**：List[bool]

#### `env_method(method_name, *method_args, indices=None, **method_kwargs)`

- 调用向量化环境实例方法。
- **参数**：
  - `method_name (str)`：要调用的方法名称。
  - `indices (None | int | Iterable[int])`：要调用方法的环境索引。
- **返回类型**：List[Any]

#### `get_attr(attr_name, indices=None)`

- 从向量化环境中返回属性。
- **参数**：
  - `attr_name (str)`：属性名称。
  - `indices (None | int | Iterable[int])`：要获取属性的环境索引。
- **返回类型**：List[Any]

#### `get_images()`

- 返回每个环境的RGB图像（如果可用）。
- **返回类型**：Sequence[ndarray | None]

#### `render(mode=None)`

- 渲染Gym环境。如果有多个环境，则通过`BaseVecEnv.render()`将它们在一个图像中平铺显示。
- **参数**：
  - `mode (str | None)`：渲染类型。
- **返回类型**：ndarray | None

#### `reset()`

- 重置所有环境，并返回一个观察数组或观察数组的元组。
- **返回类型**：ndarray | Dict[str, ndarray] | Tuple[ndarray, …]

#### `set_attr(attr_name, value, indices=None)`

- 在向量化环境内设置属性。
- **参数**：
  - `attr_name (str)`：属性名称。
  - `value (Any)`：要设置的值。
  - `indices (None | int | Iterable[int])`：要设置属性的环境索引。
- **返回类型**：None

#### `step_async(actions)`

- 告诉所有环境开始执行给定动作的步骤。使用`step_wait()`来获取步骤的结果。
- **参数**：
  - `actions (ndarray)`：动作数组。
- **返回类型**：None

#### `step_wait()`

- 等待`step_async()`执行的步骤完成。
- **返回类型**：Tuple[ndarray | Dict[str, ndarray] | Tuple[ndarray, …], ndarray, ndarray, List[Dict]]

`DummyVecEnv` 通过简化并行环境的管理和操作，使得在单个进程中同时运行多个环境成为可能。这对于在资源受限的情况下进行强化学习研究或开发非常有用，尤其是当环境计算不是主要瓶颈时。

让我们通过一个具体的代码示例来演示如何使用`DummyVecEnv`类来向量化多个环境实例。在这个示例中，我们将创建几个`CartPole-v1`环境的实例，并在这些环境上执行一系列随机动作。

### 示例代码

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np

# 定义环境创建函数
def make_env():
    return gym.make('CartPole-v1')

# 创建4个CartPole环境的列表
env_fns = [make_env for _ in range(4)]

# 使用DummyVecEnv向量化环境
vec_env = DummyVecEnv(env_fns)

# 重置环境，获取初始观察值
obs = vec_env.reset()

# 进行10个时间步的模拟
for _ in range(10):
    # 为每个环境随机生成动作
    actions = np.array([env.action_space.sample() for env in vec_env.envs])
    # 在环境上执行这些动作，并获取结果
    obs, rewards, dones, infos = vec_env.step(actions)
    # 输出奖励
    print(f"Rewards: {rewards}")

# 关闭环境
vec_env.close()
```

### 代码解释

1. **环境创建函数**:
   - `make_env`函数定义了如何创建一个新的`CartPole-v1`环境实例。这个函数在每次调用时都会返回一个新的环境实例。

2. **向量化环境**:
   - `env_fns`是一个列表，包含了几个`make_env`函数的引用。这里我们创建了4个`CartPole-v1`环境。
   - `DummyVecEnv(env_fns)`通过传入一个环境函数列表来向量化这些环境。`DummyVecEnv`将顺序调用每个函数以创建独立的环境实例，并将它们包装在一个单一的向量化环境中。

3. **环境交互**:
   - `vec_env.reset()`重置所有向量化的环境，并返回每个环境的初始观察值。
   - 在一个循环中，我们为每个环境生成一个随机动作，并通过`vec_env.step(actions)`在所有环境上执行这些动作。`step`方法返回观察值、奖励、完成标志（指示每个环境是否结束了当前episode）和额外信息。
   - 打印每一步的奖励以观察环境的反馈。

4. **环境关闭**:
   - `vec_env.close()`用于在结束模拟后清理环境资源。

这个示例展示了如何使用`DummyVecEnv`来同时管理和交互多个环境实例，这在进行并行数据收集或训练多个实例以提高学习效率时非常有用。通过这种方式，可以简化环境管理的复杂性，使得在单个进程中同时运行和监控多个环境成为可能。

## SubprocVecEnv

`SubprocVecEnv` 是 Stable Baselines3 库中用于向量化多个环境并在多个进程中分布运行每个环境的一个类。这允许在环境计算复杂时显著加速训练过程。与 `DummyVecEnv` 相比，`SubprocVecEnv` 通过使用多进程而不是在单个进程中顺序执行环境来提供性能提升，特别适用于计算密集型环境。

### 参数

- `env_fns (List[Callable[[], Env]])`：一个函数列表，每个函数在调用时创建并返回一个环境实例。
- `start_method (str | None)`：用于启动子进程的方法。必须是 `multiprocessing.get_all_start_methods()` 返回的方法之一。默认情况下，在可用的平台上为 `'forkserver'`，否则为 `'spawn'`。

### 方法和功能

#### `close()`

- 清理环境的资源。
- **返回类型**：None

#### `env_is_wrapped(wrapper_class, indices=None)`

- 检查工作环境是否被给定的包装器包装。
- **参数**：
  - `wrapper_class (Type[Wrapper])`：包装器类。
  - `indices (None | int | Iterable[int])`：要检查的环境索引。
- **返回类型**：List[bool]

#### `env_method(method_name, *method_args, indices=None, **method_kwargs)`

- 调用向量化环境实例方法。
- **参数**：
  - `method_name (str)`：要调用的方法名称。
  - `indices (None | int | Iterable[int])`：要调用方法的环境索引。
- **返回类型**：List[Any]

#### `get_attr(attr_name, indices=None)`

- 从向量化环境中返回属性。
- **参数**：
  - `attr_name (str)`：属性名称。
  - `indices (None | int | Iterable[int])`：要获取属性的环境索引。
- **返回类型**：List[Any]

#### `get_images()`

- 返回每个环境的RGB图像（如果可用）。
- **返回类型**：Sequence[ndarray | None]

#### `reset()`

- 重置所有环境，并返回一个观察数组或观察数组的元组。
- **返回类型**：ndarray | Dict[str, ndarray] | Tuple[ndarray, …]

#### `set_attr(attr_name, value, indices=None)`

- 在向量化环境内设置属性。
- **参数**：
  - `attr_name (str)`：属性名称。
  - `value (Any)`：要设置的值。
  - `indices (None | int | Iterable[int])`：要设置属性的环境索引。
- **返回类型**：None

#### `step_async(actions)`

- 告诉所有环境开始执行给定动作的步骤。使用 `step_wait()` 来获取步骤的结果。
- **参数**：
  - `actions (ndarray)`：动作数组。
- **返回类型**：None

#### `step_wait()`

- 等待 `step_async()` 执行的步骤完成。
- **返回类型**：Tuple[ndarray | Dict[str, ndarray] | Tuple[ndarray, …], ndarray, ndarray, List[Dict]]

`SubprocVecEnv` 通过在多个进程中并行运行环境来提高训练速度，特别适合于那些计算密集型的环境。使用这个类可以有效地利用多核CPU的计算能力，加快数据收集和训练过程。

让我们通过一个具体的代码示例来演示如何使用`SubprocVecEnv`类在多个进程中向量化和运行多个环境实例。这个示例将创建几个`CartPole-v1`环境的实例，并在这些环境上执行一系列随机动作。

### 示例代码

```python
import gym
from stable_baselines3.common.vec_env import SubprocVecEnv
import numpy as np

# 定义环境创建函数
def make_env(env_id, seed):
    def _init():
        env = gym.make(env_id)
        env.seed(seed)
        return env
    return _init

env_id = "CartPole-v1"  # 环境ID
num_envs = 4  # 并行环境的数量

# 创建环境函数列表
env_fns = [make_env(env_id, seed=i) for i in range(num_envs)]

# 使用SubprocVecEnv向量化环境
vec_env = SubprocVecEnv(env_fns)

# 重置环境，获取初始观察值
obs = vec_env.reset()

# 进行10个时间步的模拟
for _ in range(10):
    # 为每个环境随机生成动作
    actions = np.array([vec_env.action_space.sample() for _ in range(num_envs)])
    # 在环境上执行这些动作，并获取结果
    obs, rewards, dones, infos = vec_env.step(actions)
    # 输出奖励
    print(f"Rewards: {rewards}")

# 关闭环境
vec_env.close()
```

### 代码解释

1. **环境创建函数**:
   - `make_env`函数定义了如何创建一个新的`CartPole-v1`环境实例。每个环境被赋予了一个唯一的种子（通过`seed`参数），以确保环境的随机性。

2. **向量化环境**:
   - `env_fns`是一个包含环境创建函数的列表，每个函数都配置了一个不同的种子值。
   - `SubprocVecEnv(env_fns)`通过这些函数在不同的进程中创建并向量化了几个环境实例。这允许每个环境实例独立运行在自己的进程中，从而利用多核心CPU的优势。

3. **环境交互**:
   - `vec_env.reset()`重置所有向量化的环境，并返回每个环境的初始观察值。
   - 在一个循环中，我们为每个环境生成一个随机动作，并通过`vec_env.step(actions)`在所有环境上执行这些动作。`step`方法返回观察值、奖励、完成标志（指示每个环境是否结束了当前episode）和额外信息。

4. **环境关闭**:
   - `vec_env.close()`用于在结束模拟后清理环境资源。

这个示例展示了如何使用`SubprocVecEnv`来在多个进程中并行运行多个环境实例，这对于加速数据收集和训练过程非常有用。通过这种方式，可以有效地利用多核CPU的计算能力，从而提高整体的训练效率。

# 包装器Wrappers

## VecFrameStack

`VecFrameStack` 是 Stable Baselines3 库中的一个向量化环境包装器，用于在向量化环境上实现帧堆叠功能。这对于图像观察特别有用，因为通过堆叠连续的帧，代理可以从序列中学习更多的时间动态信息，这在处理视频游戏或任何需要理解时间连续性的场景中特别重要。

### 参数

- `venv (VecEnv)`: 要包装的向量化环境。
- `n_stack (int)`: 要堆叠的帧数。
- `channels_order (str | Mapping[str, str] | None)`: 帧堆叠的维度顺序。如果为"first"，则在图像的第一个维度（通道维度）上堆叠；如果为"last"，则在最后一个维度上堆叠。如果为`None`，则自动检测在图像观察情况下的堆叠维度，或默认使用"last"。`channels_order`也可以是字典，这在处理具有字典观察空间的环境时特别有用。

### 方法

#### `reset()`

- 重置所有环境。在这个过程中，它还会重置帧堆叠。
- **返回类型**: `ndarray | Dict[str, ndarray]`，返回重置后的观察值，这些观察值将包含堆叠的帧。

#### `step_wait()`

- 等待`step_async()`方法执行的步骤完成，并返回堆叠的帧作为观察结果。
- **返回**: `Tuple[ndarray | Dict[str, ndarray], ndarray, ndarray, List[Dict[str, Any]]]`，其中包括堆叠的观察结果、奖励、完成标志以及额外信息。

### 使用示例

假设我们有一个返回图像观察的环境，我们想要堆叠最近的4帧：

```python
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import gym

# 创建向量化环境
env = DummyVecEnv([lambda: gym.make("Breakout-v0")])

# 对环境应用帧堆叠包装器
env = VecFrameStack(env, n_stack=4)

# 重置环境
obs = env.reset()

# 检查返回的观察值形状，应该反映出堆叠的帧
print("Observation shape:", obs.shape)

# 执行一步动作，观察返回的观察值
obs, rewards, dones, infos = env.step([env.action_space.sample()])

# 再次检查观察值形状
print("Observation shape after one step:", obs.shape)
```

在这个示例中，`VecFrameStack` 包装器被用来在`DummyVecEnv`上堆叠4帧。通过堆叠帧，观察值的形状将扩展以包含额外的时间维度（根据`channels_order`参数在图像的开始或结束处）。这使得代理能够观察到关于环境动态的更多信息，从而在需要理解时间连续性的任务中做出更好的决策。

让我们通过一个具体的代码示例来展示如何使用`VecFrameStack`类在向量化环境中实现帧堆叠功能。这个示例将创建一个`Pong-v0`环境的向量化实例，并应用帧堆叠，以便代理能够从连续帧的序列中学习。

### 示例代码

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import numpy as np

# 定义环境创建函数
def make_env(env_name):
    def _init():
        return gym.make(env_name)
    return _init

env_name = "PongNoFrameskip-v4"  # 使用Pong游戏作为示例
num_envs = 1  # 在这个示例中，我们只创建一个环境的实例

# 使用DummyVecEnv创建向量化环境
vec_env = DummyVecEnv([make_env(env_name) for _ in range(num_envs)])

# 使用VecFrameStack对环境应用帧堆叠包装器，堆叠4帧
stacked_env = VecFrameStack(vec_env, n_stack=4)

# 重置环境，开始新的观察
obs = stacked_env.reset()

# 检查堆叠后的观察值形状
print("Stacked observation shape:", obs.shape)

# 执行一些随机动作，观察堆叠帧的变化
for _ in range(5):
    actions = [stacked_env.action_space.sample() for _ in range(num_envs)]
    obs, rewards, dones, infos = stacked_env.step(actions)
    print("Rewards:", rewards)
    # 注意：在这里我们不打印观察值，因为它们是图像数据，但你可以使用图像处理库（如matplotlib）来可视化它们

# 关闭环境
stacked_env.close()
```

### 代码解释

1. **环境创建**:
   - 使用`make_env`函数来定义如何创建Pong游戏的环境实例。这个函数在调用时会创建并返回一个新的Gym环境实例。

2. **向量化和帧堆叠**:
   - 首先，使用`DummyVecEnv`将创建的环境包装成一个向量化环境。尽管我们在这个示例中只使用了一个环境实例，但`DummyVecEnv`和`VecFrameStack`可以轻松地扩展到多个并行环境。
   - 然后，使用`VecFrameStack`对向量化环境进行包装，以在环境的观察中堆叠连续的4帧。这允许代理从图像序列中获取更多的时间动态信息。

3. **环境交互**:
   - 在重置环境后，我们通过执行随机动作来与环境进行交互，并观察结果。每次`step`调用后，我们都会获得一个包含堆叠帧的新观察值。

4. **观察值形状**:
   - 打印堆叠后的观察值形状，可以看到观察值的维度已经增加，以反映额外的堆叠帧。在`PongNoFrameskip-v4`的情况下，原始观察是单帧图像，而堆叠后的观察值则包含4个连续帧的数据。

这个示例展示了如何通过`VecFrameStack`在向量化环境中实现帧堆叠，这对于处理需要理解环境内时间连续性的任务非常有用，如视频游戏。帧堆叠使代理能够观察到过去的状态，这对于学习如何根据过去和当前的信息做出决策是非常有价值的。

## 堆叠观察空间StackedObservations

`StackedObservations`是Stable Baselines3库中用于数据帧堆叠的一个包装器。它不直接应用于环境，而是作为向量化环境中处理帧堆叠逻辑的内部机制。`StackedObservations`可以自动处理观察空间中的图像数据，根据图像的维度（通道在前或通道在后）来决定如何堆叠帧。

### 参数

- `num_envs`: 环境的数量。
- `n_stack`: 要堆叠的帧数。
- `observation_space`: 环境的观察空间。
- `channels_order`: 如果是"first"，则在图像的第一个维度上堆叠；如果是"last"，则在最后一个维度上堆叠。如果为`None`，则会自动检测在图像观察情况下的堆叠维度，默认使用"last"。对于字典空间，`channels_order`也可以是一个字典。

### 方法

#### `compute_stacking(n_stack, observation_space, channels_order=None)`

- 计算堆叠观察值所需的参数。
- **参数**:
  - `n_stack (int)`: 要堆叠的观察数量。
  - `observation_space (Box)`: 观察空间。
  - `channels_order (str | None)`: 通道的顺序。
- **返回**: 通道是否在前、堆叠维度、堆叠后的观察形状、重复轴的元组。

#### `reset(observation)`

- 重置堆叠的观察值，添加重置观察到堆栈中，并返回堆栈。
- **参数**:
  - `observation (TObs)`: 重置观察。
- **返回**: 堆叠的重置观察。

#### `update(observations, dones, infos)`

- 将观察值添加到堆栈中，并使用完成标志（dones）更新信息（infos）。
- **参数**:
  - `observations (TObs)`: 观察值。
  - `dones (ndarray)`: 完成标志。
  - `infos (List[Dict[str, Any]])`: 信息。
- **返回**: 堆叠的观察值和更新后的信息的元组。

`StackedObservations`通过自动管理多个帧的堆叠，为基于图像的强化学习任务提供了方便。这种堆叠机制使得代理能够从连续的帧中学习时间序列信息，这对于理解动态环境中的时间依赖性至关重要。通过使用这个工具，可以轻松地在训练过程中引入额外的时间维度，而不需要手动管理每个时间步的观察值堆叠。

`StackedObservations` 类在 Stable Baselines3 的公开接口中不直接使用，而是作为实现帧堆叠功能的底层机制，如在 `VecFrameStack` 包装器中使用。然而，了解其如何工作可以帮助我们更好地理解帧堆叠背后的逻辑。

下面我将提供一个理论上的使用示例，模拟`StackedObservations`的工作原理，并解释如何在自定义实现中使用它。请注意，这不是一个实际的代码示例，因为在 Stable Baselines3 中，你通常会使用 `VecFrameStack` 或其他高级接口来实现帧堆叠。

### 模拟`StackedObservations`的工作原理

假设我们有一个环境，其观察空间是一个图像，我们想要手动实现一个简单的帧堆叠机制。

1. **初始化**: 假设观察空间是一个84x84的灰度图像（单通道），我们想要堆叠4帧。

2. **重置环境**: 在环境重置时，我们获取初始帧，并将其复制4次来初始化堆叠。

3. **环境步进**: 在每个时间步，我们获取新的观察值，并将其添加到堆叠中，同时移除最旧的帧。

### 代码示例

```python
import numpy as np

class SimpleStackedObservations:
    def __init__(self, num_envs, n_stack, observation_shape, channels_order='last'):
        self.num_envs = num_envs
        self.n_stack = n_stack
        self.observation_shape = observation_shape
        self.channels_order = channels_order
        # 根据channels_order初始化堆叠的观察形状
        if channels_order == 'last':
            self.stacked_obs = np.zeros((num_envs,) + observation_shape + (n_stack,), dtype=np.float32)
        else:
            self.stacked_obs = np.zeros((num_envs, n_stack) + observation_shape, dtype=np.float32)

    def reset(self, observation):
        # 重置时复制初始帧来填充堆叠
        if self.channels_order == 'last':
            self.stacked_obs[..., :] = np.repeat(observation[..., np.newaxis], self.n_stack, axis=-1)
        else:
            self.stacked_obs[:, :, ...] = np.repeat(observation[np.newaxis, ...], self.n_stack, axis=1)
        return self.stacked_obs

    def update(self, observation):
        # 将新帧添加到堆叠中，并移除最旧的帧
        self.stacked_obs = np.roll(self.stacked_obs, shift=-1, axis=-1)
        if self.channels_order == 'last':
            self.stacked_obs[..., -1] = observation
        else:
            self.stacked_obs[:, -1, ...] = observation
        return self.stacked_obs

# 假设的初始化参数
num_envs = 1
n_stack = 4
observation_shape = (84, 84)  # 灰度图像，84x84
channels_order = 'last'  # 假设图像通道在最后

# 创建一个简单的堆叠观察对象
stacker = SimpleStackedObservations(num_envs, n_stack, observation_shape, channels_order)

# 模拟环境重置
initial_observation = np.random.rand(84, 84)  # 随机生成初始观察
stacked_obs = stacker.reset(initial_observation)

# 模拟环境步进
for _ in range(10):  # 模拟10个时间步
    new_observation = np.random.rand(84, 84)  # 生成新观察
    stacked_obs = stacker.update(new_observation)
    # 此处可以使用stacked_obs进行决策
```

这个简化的示例展示了如何手动实现观察值的帧堆叠机制。在实际应用中，Stable Baselines3 的 `VecFrameStack` 包装器或类似机制会处理这

些细节，使得在强化学习训练中轻松地利用时间序列信息。

## 向量归一化VecNormalize

`VecNormalize` 是 Stable Baselines3 库中用于向量化环境的一个包装器，它通过维护移动平均值来对观察和奖励进行标准化处理。这个包装器特别适用于处理那些观察值或奖励范围变化很大的环境，通过标准化可以帮助加快学习过程和提高算法的稳定性。

### 参数

- `venv (VecEnv)`: 要包装的向量化环境。
- `training (bool)`: 是否更新移动平均。在训练时通常设置为True，评估时设置为False。
- `norm_obs (bool)`: 是否对观察进行标准化，默认为True。
- `norm_reward (bool)`: 是否对奖励进行标准化，默认为True。
- `clip_obs (float)`: 观察值的最大绝对值。
- `clip_reward (float)`: 奖励的最大绝对值。
- `gamma (float)`: 折扣因子，用于计算折扣奖励。
- `epsilon (float)`: 避免除零的小值。
- `norm_obs_keys (List[str] | None)`: 从观察字典中哪些键需要标准化。如果未指定，则标准化所有键。

### 方法

#### `get_original_obs()`

- 返回最近一步或重置时的未标准化观察值。
- **返回类型**: `ndarray | Dict[str, ndarray]`

#### `get_original_reward()`

- 返回最近一步的未标准化奖励。
- **返回类型**: `ndarray`

#### `normalize_obs(obs)`

- 使用这个`VecNormalize`的观察统计数据标准化观察值。调用此方法不会更新统计数据。
- **参数**: `obs (ndarray | Dict[str, ndarray])`
- **返回类型**: `ndarray | Dict[str, ndarray]`

#### `normalize_reward(reward)`

- 使用这个`VecNormalize`的奖励统计数据标准化奖励。调用此方法不会更新统计数据。
- **参数**: `reward (ndarray)`
- **返回类型**: `ndarray`

#### `reset()`

- 重置所有环境，并返回首个观察值。
- **返回类型**: `ndarray | Dict[str, ndarray]`

#### `save(save_path)`

- 保存当前`VecNormalize`对象及其所有运行统计数据和设置。
- **参数**: `save_path (str)`

#### `set_venv(venv)`

- 设置要包装的向量环境。
- **参数**: `venv (VecEnv)`

#### `step_wait()`

- 对环境序列应用一系列动作，返回观察、奖励和完成标志。
- **返回类型**: `Tuple[ndarray | Dict[str, ndarray] | Tuple[ndarray, …], ndarray, ndarray, List[Dict]]`

### 使用示例

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3 import PPO

# 创建向量化环境
env = gym.make('CartPole-v1')
vec_env = DummyVecEnv([lambda: env])

# 应用VecNormalize包装器
norm_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)

# 创建模型
model = PPO("MlpPolicy", norm_env, verbose=1)

# 训练模型
model.learn(total_timesteps=10000)

# 保存标准化环境的统计数据
norm_env.save("vec_normalize.pkl")

# 加载标准化统计数据
loaded_env = VecNormalize.load("vec_normalize.pkl", vec_env)

# 在加载的环境上评估模型
obs = loaded_env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = loaded_env.step(action)
    if dones:
        break
```

在这个示例中，`VecNormalize` 被用来标准化`CartPole-v1`环境的观察值和奖励。通过在训练和评估过程中使用相同的标准化参数，可以保证模型的表现与训练时相似。这对于提高训练过程的效率和稳定性非常有帮助。

下面的示例代码展示了如何使用`VecNormalize`类来标准化观测和奖励，以及如何在训练和评估阶段正确地保存和加载标准化环境的统计信息。这个过程对于确保模型在评估或实际应用时能够接收到与训练期间相同分布的输入是非常重要的。

### 示例代码

```python
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# 确保保存路径存在
save_dir = "./saved_models/"
os.makedirs(save_dir, exist_ok=True)

# 创建和包装环境
env = gym.make('CartPole-v1')
vec_env = DummyVecEnv([lambda: env])  # 将环境向量化
norm_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True, clip_obs=10.0, clip_reward=10.0)  # 应用标准化

# 创建模型
model = PPO("MlpPolicy", norm_env, verbose=1)

# 训练模型
model.learn(total_timesteps=20000)

# 保存模型和标准化环境的统计信息
model.save(os.path.join(save_dir, "ppo_cartpole"))
norm_env.save(os.path.join(save_dir, "vecnormalize.pkl"))

# 清理
norm_env.close()

# 加载模型和标准化环境的统计信息
loaded_model = PPO.load(os.path.join(save_dir, "ppo_cartpole"))

# 在新的环境实例上加载标准化统计信息
test_env = gym.make('CartPole-v1')
test_vec_env = DummyVecEnv([lambda: test_env])
test_norm_env = VecNormalize.load(os.path.join(save_dir, "vecnormalize.pkl"), test_vec_env)

test_norm_env.training = False  # 确保在评估时不更新统计信息
test_norm_env.norm_reward = False  # 通常在评估时不标准化奖励

# 评估模型
obs = test_norm_env.reset()
for _ in range(1000):
    action, _states = loaded_model.predict(obs, deterministic=True)
    obs, reward, done, info = test_norm_env.step(action)
    if done:
        break

test_norm_env.close()
```

### 代码解释

1. **创建和包装环境**：
   - 使用`gym.make`创建一个CartPole-v1环境实例。
   - 使用`DummyVecEnv`将环境向量化，这允许Stable Baselines3的模型与环境交互。
   - 使用`VecNormalize`包装向量化的环境，以标准化观测和奖励。这里设置了观测和奖励的最大剪裁值。

2. **训练模型**：
   - 创建一个PPO模型，指定策略网络为"MlpPolicy"，并开始训练。

3. **保存模型和标准化环境统计信息**：
   - 训练完成后，保存模型和标准化环境的统计信息到文件。这些统计信息对于评估和部署模型时保持输入数据的一致性至关重要。

4. **加载模型和标准化环境统计信息**：
   - 为了评估，我们加载之前保存的模型和标准化环境统计信息。
   - 创建一个新的环境实例，并用保存的统计信息初始化一个新的`VecNormalize`实例。

5. **评估模型**：
   - 在评估阶段，通过将`training`设置为`False`来确保不更新标准化的统计信息，并且通常不需要标准化奖励。
   - 然后在新的标准化环境中运行模型，执行动作并观察模型的表现。

此示例展示了如何在Stable Baselines3中使用`VecNormalize`来标准化环境的观测和奖

励，并展示了如何在训练和评估阶段保存和加载标准化的统计信息，以确保模型的稳定和高效学习。

## VecVideoRecorder

`VecVideoRecorder`是Stable Baselines3库中的一个包装器，用于在训练过程中记录向量化环境的视频。这个功能对于可视化代理学习过程和评估其表现特别有用。要使用这个包装器，系统上需要安装ffmpeg或avconv。

### 参数

- `venv (VecEnv)`: 要记录视频的向量化环境或环境包装器。
- `video_folder (str)`: 视频文件保存的目录。
- `record_video_trigger (Callable[[int], bool])`: 决定何时开始录制视频的函数。该函数接受当前步骤数作为输入，并返回一个布尔值，指示是否应该开始录制。
- `video_length (int)`: 记录的视频长度，以时间步为单位。
- `name_prefix (str)`: 视频文件名的前缀。

### 方法

#### `close()`

- 清理环境资源。在不再需要录制视频时调用。

#### `reset()`

- 重置所有环境，并返回初始观察数组。这通常在开始新的episode时调用。

#### `step_wait()`

- 等待`step_async()`执行的动作完成。返回观察值、奖励、完成标志和额外信息。

### 使用示例

下面的代码展示了如何使用`VecVideoRecorder`来记录一个简单的CartPole环境的视频。

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
from stable_baselines3 import A2C

def record_video_trigger(step: int) -> bool:
    """每1000步记录一次视频"""
    return step % 1000 == 0

# 创建环境并向量化
env = gym.make('CartPole-v1')
vec_env = DummyVecEnv([lambda: env])

# 创建视频记录器
video_folder = './videos'
video_length = 200
vec_env = VecVideoRecorder(vec_env, video_folder, record_video_trigger, video_length)

# 创建模型
model = A2C("MlpPolicy", vec_env, verbose=1)

# 训练模型，同时记录视频
model.learn(total_timesteps=5000)

# 关闭环境和视频记录器
vec_env.close()
```

在这个示例中，我们首先创建了一个CartPole环境，并使用`DummyVecEnv`进行向量化。然后，我们使用`VecVideoRecorder`包装器来包装环境，指定了视频保存的位置、何时开始录制的条件、视频长度和文件名前缀。在模型训练过程中，根据`record_video_trigger`函数定义的条件，会自动开始录制视频，并保存到指定的目录中。

这个功能特别适用于监控和评估代理的学习进程，以及生成展示材料。

让我们通过一个详细的示例来展示如何使用`VecVideoRecorder`类在训练过程中自动记录视频。这个示例将使用OpenAI Gym中的`LunarLander-v2`环境，使用PPO算法进行训练，并且每隔一定时间步自动记录一个视频。

### 环境准备

首先，确保你的环境中已经安装了必要的库：

```sh
pip install stable-baselines3 gym
```

同时，确保你的系统上已经安装了`ffmpeg`，因为`VecVideoRecorder`需要`ffmpeg`来编码视频文件。

### 示例代码

```python
import os
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecVideoRecorder
import gym

# 创建视频记录的触发器函数
def record_video_trigger(step):
    return step % 1000 == 0  # 每1000步记录一个视频

# 创建和向量化环境
env_id = "LunarLander-v2"
vec_env = make_vec_env(env_id, n_envs=1)

# 定义视频保存路径和视频长度
video_folder = "./videos"
video_length = 500
name_prefix = "ppo-lunarlander"

# 创建VecVideoRecorder包装器实例
video_env = VecVideoRecorder(vec_env, video_folder,
                             record_video_trigger,
                             video_length=video_length,
                             name_prefix=name_prefix)

# 创建模型
model = PPO("MlpPolicy", video_env, verbose=1)

# 训练模型
total_timesteps = 5000
model.learn(total_timesteps=total_timesteps)

# 保存模型
model.save("ppo_lunarlander")

# 关闭环境
video_env.close()

# 如果需要，可以加载模型并继续训练或评估
# model = PPO.load("ppo_lunarlander", env=video_env)
```

### 代码解释

1. **视频记录触发器**：定义了一个简单的函数`record_video_trigger`，它决定了何时开始录制视频。在这个示例中，我们选择每1000个时间步记录一个视频。

2. **环境创建与向量化**：使用`make_vec_env`函数创建并向量化`LunarLander-v2`环境。`n_envs=1`表示我们只使用一个环境实例。

3. **视频记录环境包装器**：通过`VecVideoRecorder`将向量化环境包装起来，指定视频保存的文件夹、何时记录视频的触发器、视频长度以及视频文件名前缀。

4. **模型创建与训练**：使用PPO算法创建模型，并在包装过的环境上训练指定的时间步。在训练过程中，根据触发器函数的条件，会自动记录视频。

5. **模型保存**：训练完成后，模型被保存到磁盘上。这允许后续加载模型进行进一步的训练或评估。

6. **环境关闭**：训练完成后，通过调用`video_env.close()`来关闭环境和释放资源。

通过这个示例，你可以看到如何在训练过程中使用`VecVideoRecorder`自动记录视频，这对于可视化代理的学习过程、分析行为或生成展示材料非常有用。

## VecCheckNan

`VecCheckNan`是Stable Baselines3库中的一个向量化环境包装器，用于监测环境中的NaN和无穷值。当环境的观察值、奖励或信息包含这些数值时，它能够提醒用户，帮助诊断和修复潜在的问题。

### 参数

- `venv (VecEnv)`: 要包装的向量化环境。
- `raise_exception (bool)`: 如果为True，当检测到NaN或无穷值时将引发`ValueError`异常；否则，默认行为是发出警告。
- `warn_once (bool)`: 如果为True，对于每种类型的问题只警告一次，避免日志被重复警告信息淹没。
- `check_inf (bool)`: 是否检查正无穷和负无穷值。默认为True。

### 方法

#### `check_array_value(name, value)`

- 检查单个numpy数组中的NaN和无穷值。
- **参数**:
  - `name (str)`: 正在检查的值的名称，用于在警告或异常消息中标识来源。
  - `value (ndarray)`: 需要检查的numpy数组。
- **返回**: 发现的问题列表，每个问题是一个包含问题类型和描述的元组。

#### `reset()`

- 重置所有环境，并返回初始观察值。在重置过程中，也会检查返回的观察值是否包含NaN或无穷值。
- **返回类型**: `ndarray | Dict[str, ndarray] | Tuple[ndarray, …]`

#### `step_async(actions)`

- 通知所有环境开始根据给定动作执行步骤。
- **参数**:
  - `actions (ndarray)`: 动作数组。

#### `step_wait()`

- 等待`step_async()`执行的步骤完成，并检查结果中的观察值、奖励和信息是否包含NaN或无穷值。
- **返回类型**: `Tuple[ndarray | Dict[str, ndarray] | Tuple[ndarray, …], ndarray, ndarray, List[Dict]]`

### 使用示例

下面的代码演示了如何将`VecCheckNan`包装器应用于一个简单的环境，并处理可能发生的NaN值。

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecCheckNan

# 创建一个环境实例
env = gym.make('CartPole-v1')
# 向量化环境
vec_env = DummyVecEnv([lambda: env])
# 应用VecCheckNan包装器
nan_check_env = VecCheckNan(vec_env, raise_exception=True)

# 尝试重置环境和执行步骤，监控NaN值
try:
    obs = nan_check_env.reset()
    for _ in range(10):
        action = [nan_check_env.action_space.sample()]
        obs, rewards, dones, infos = nan_check_env.step(action)
        if dones[0]:
            break
except ValueError as e:
    print(f"Caught ValueError: {e}")
```

在这个示例中，我们首先创建了一个`CartPole-v1`环境并将其向量化。然后，我们通过`VecCheckNan`包装器来包装向量化的环境，设置`raise_exception=True`以在检测到NaN值时引发异常。这样，在训练循环中，如果环境的返回值包含NaN或无穷值，我们将立即得到通知，并能够采取适当的修复措施。

让我们通过一个完整的代码示例来展示如何使用`VecCheckNan`类来监测和处理环境中的NaN值和无穷大值。这个示例将使用一个假设的环境，我们将在其中人为地引入NaN值，以演示`VecCheckNan`的工作原理和如何在实际中使用它。

### 示例环境定义

首先，我们定义一个简单的环境，该环境在某些情况下会返回NaN观察值。为了简化，我们直接使用OpenAI Gym的`CartPole-v1`环境，并修改其`step`函数。

```python
import gym
import numpy as np

class NanCartPoleEnv(gym.Env):
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        self.step_counter = 0

    def step(self, action):
        self.step_counter += 1
        obs, reward, done, info = self.env.step(action)
        # 每5步返回一个包含NaN的观察值
        if self.step_counter % 5 == 0:
            obs = np.array([np.nan, np.nan, np.nan, np.nan])
        return obs, reward, done, info

    def reset(self):
        self.step_counter = 0
        return self.env.reset()

    def render(self, mode='human'):
        self.env.render(mode=mode)

    def close(self):
        self.env.close()
```

### 应用VecCheckNan

现在，我们将`NanCartPoleEnv`环境包装在`VecCheckNan`中，并尝试执行一些步骤以观察`VecCheckNan`如何检测到NaN值并发出警告或异常。

```python
from stable_baselines3.common.vec_env import DummyVecEnv, VecCheckNan

# 创建NanCartPole环境的实例并向量化
vec_env = DummyVecEnv([lambda: NanCartPoleEnv()])

# 使用VecCheckNan包装向量化的环境，设置为在检测到NaN时引发异常
check_nan_env = VecCheckNan(vec_env, raise_exception=True)

try:
    # 重置环境
    obs = check_nan_env.reset()
    # 尝试执行一些步骤
    for _ in range(10):
        action = [check_nan_env.action_space.sample()]
        obs, rewards, dones, infos = check_nan_env.step(action)
        print(f"Step {_}: Observation: {obs}")
except ValueError as e:
    print(f"Exception caught: {e}")

# 关闭环境
check_nan_env.close()
```

### 代码解释

- 我们定义了一个`NanCartPoleEnv`类，该类基于标准的`CartPole-v1`环境，但在每5个步骤中人为地引入一个包含NaN值的观察值。
- 使用`DummyVecEnv`来向量化我们的自定义环境，这允许`VecCheckNan`工作在向量化环境上。
- `VecCheckNan`被设置为`raise_exception=True`，这意味着当检测到NaN观察值时，它将引发一个`ValueError`异常而不是仅仅发出警告。
- 在尝试执行环境步骤的循环中，当我们达到人为引入NaN的步骤时，`VecCheckNan`将检测到NaN观察值并引发异常，我们通过`try-except`结构捕获这个异常并打印出相关信息。
- 最后，我们确保在结束时关闭环境。

这个示例展示了`VecCheckNan`如何在实际应用中用于监测和处理向量化环境中的NaN值和无穷大值，帮助开发者诊断和修复可能导致这些数值出现的问题。

## 向量转置图像VecTransposeImage

`VecTransposeImage` 是 Stable Baselines3 库中的一个向量化环境包装器，用于将环境观察值中的图像通道顺序从高度x宽度x通道（HxWxC）调整为通道x高度x宽度（CxHxW）。这种转换对于使用PyTorch构建的卷积神经网络是必要的，因为PyTorch卷积层默认期望图像通道在前。

### 参数

- `venv (VecEnv)`: 要包装的向量化环境。
- `skip (bool)`: 如果需要跳过这个包装器，则设置为True。这个参数通常基于是否已经手动调整了图像数据的格式，或者基于特定环境的需求而设置。

### 方法

#### `close()`

- 清理环境资源。在环境不再使用时调用。

#### `reset()`

- 重置所有环境，返回重置后的观察值。重置后的观察值会根据需要进行转置。

#### `step_wait()`

- 等待`step_async()`执行的动作完成，返回观察值、奖励、完成标志和额外信息。返回的观察值会根据需要进行转置。

#### `transpose_image(image)`

- 转置单个图像或图像批次。这个方法是静态的，可以直接调用来转置图像数据。

#### `transpose_observations(observations)`

- 转置观察值（如果需要），并返回新的观察值。适用于处理批次观察值的情况。

#### `transpose_space(observation_space, key='')`

- 转置观察空间（重新排序通道）。这对于调整环境观察空间的定义以匹配图像数据的转置格式是必要的。

### 使用示例

以下示例展示了如何将`VecTransposeImage`应用于向量化环境，并准备它以用于PyTorch模型。

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage
from stable_baselines3 import PPO

# 假设有一个返回图像观察值的环境
env_id = "CarRacing-v0"
# 创建向量化环境
vec_env = DummyVecEnv([lambda: gym.make(env_id)])

# 应用VecTransposeImage包装器，以便图像通道顺序符合PyTorch的期望
transposed_env = VecTransposeImage(vec_env)

# 创建模型，这里以PPO为例，使用转置后的环境
model = PPO("CnnPolicy", transposed_env, verbose=1)

# 训练模型
model.learn(total_timesteps=10000)

# 注意: 实际使用时，请确保环境确实返回图像数据，并且你的PyTorch模型期望通道在前的输入格式。
```

在这个示例中，`VecTransposeImage`包装器被用来调整`CarRacing-v0`环境的观察值格式，使之适配PyTorch中卷积层的期望输入格式。这样做可以简化数据预处理步骤，使得开发者能够直接在环境观察值上应用PyTorch模型，而无需额外的数据格式调整。

下面的代码示例展示了如何将`VecTransposeImage`类应用于向量化环境，并为使用PyTorch构建的深度学习模型准备观察值。这个示例中，我们将使用OpenAI Gym的`Breakout-v0`环境，这是一个返回图像观察值的环境，通常用于测试强化学习算法。

### 准备工作

首先，请确保你已经安装了必要的库：

```sh
pip install stable-baselines3 gym[atari]
```

我们将使用PyTorch进行模型训练，也请确保安装了PyTorch。你可以从[PyTorch官网](https://pytorch.org/)找到安装指令。

### 示例代码

```python
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import torch

class CustomCNN(BaseFeaturesExtractor):
    """
    自定义的CNN特征提取器，用于处理转置后的图像数据。
    """
    def __init__(self, observation_space, features_dim=512):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # 基于通道在前的输入数据定义卷积层
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )
        # 定义后续的全连接层
        self.linear = nn.Sequential(
            nn.Linear(64 * 7 * 7, features_dim),
            nn.ReLU(),
        )

    def forward(self, observations):
        return self.linear(self.cnn(observations))

# 创建并向量化环境
env_id = "BreakoutNoFrameskip-v4"
vec_env = DummyVecEnv([lambda: gym.make(env_id)])

# 应用VecTransposeImage包装器，确保图像的通道顺序符合PyTorch的期望（通道在前）
vec_env = VecTransposeImage(vec_env)

# 使用自定义的CNN特征提取器创建PPO模型
policy_kwargs = dict(features_extractor_class=CustomCNN)
model = PPO("CnnPolicy", vec_env, verbose=1, policy_kwargs=policy_kwargs)

# 训练模型
model.learn(total_timesteps=10000)
```

### 代码解释

- **自定义CNN特征提取器**：我们定义了一个继承自`BaseFeaturesExtractor`的自定义卷积神经网络（CNN）。这个网络针对的是通道在前（CxHxW）的图像数据格式，这正是`VecTransposeImage`所进行的转换。

- **环境准备**：我们首先创建了`BreakoutNoFrameskip-v4`环境，它是Atari游戏Breakout的一个变种，然后通过`DummyVecEnv`进行向量化，以兼容Stable Baselines3的接口要求。

- **应用VecTransposeImage**：通过`VecTransposeImage`包装器，我们确保了环境观察值的通道顺序调整为CxHxW，以匹配PyTorch的期望输入格式。

- **创建模型**：在创建PPO模型时，我们通过`policy_kwargs`参数指定了我们的自定义CNN作为特征提取器。这样，模型就能够接收和处理通过`VecTransposeImage`转置后的观察值。

- **模型训练**：最后，我们训练模型一定数量的时间步。在训练过程中，模型会接收到正确格式的图像数据，并利用自定义的CNN来提取特征进行学习。

通过这个示例，你可以看到如何将`VecTransposeImage`与自

定义的PyTorch CNN结合使用，以便在使用Stable Baselines3进行强化学习训练时处理图像观察值。

## VecMonitor

`VecMonitor`是Stable Baselines3库中的一个向量化环境包装器，用于记录向量化Gym环境中的各种数据，包括每个episode的奖励、长度、时间和其他数据。这个包装器在向量化环境级别上执行原始`Monitor`包装器的功能，特别适用于那些直接初始化向量化环境的情况，如`openai/procgen`或`gym3`，在这些情况下，我们无法直接使用`Monitor`包装器。

### 参数

- `venv (VecEnv)`: 要监控的向量化环境。
- `filename (str | None)`: 保存日志文件的位置。如果为`None`，则不保存日志。
- `info_keywords (Tuple[str, ...])`: 从环境的`step()`返回的信息中额外记录的关键字信息。

### 方法

#### `close()`

- 清理环境资源。在环境不再使用时调用。

#### `reset()`

- 重置所有环境，并返回初始观察值数组。

#### `step_wait()`

- 等待`step_async()`执行的动作完成，并返回观察值、奖励、完成标志和额外信息。在此过程中，将记录相关数据。

### 使用示例

以下示例演示了如何使用`VecMonitor`来记录向量化环境的性能数据。

```python
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.env_util import make_vec_env

# 创建向量化环境
env_id = "CartPole-v1"
vec_env = make_vec_env(env_id, n_envs=4)

# 应用VecMonitor包装器，并指定日志文件的保存位置
log_file = "./vec_monitor_log"
vec_env = VecMonitor(vec_env, filename=log_file)

# 创建模型
model = PPO("MlpPolicy", vec_env, verbose=1)

# 训练模型
model.learn(total_timesteps=10000)

# 关闭环境
vec_env.close()
```

这个示例中，我们首先使用`make_vec_env`创建了一个`CartPole-v1`环境的向量化版本，并且使用`VecMonitor`包装器来包装这个向量化环境。我们指定了一个日志文件的保存位置，这样在训练过程中，每个episode的奖励、长度和时间等数据就会被记录下来。这对于后续分析模型性能和环境交互效果非常有用。训练完成后，我们关闭环境以释放资源。

让我们通过一个详细的示例来展示如何使用`VecMonitor`类对训练过程中的关键数据进行记录。这个示例将使用OpenAI Gym的`LunarLander-v2`环境，一个常用于测试强化学习算法的环境。我们将使用PPO算法进行训练，并利用`VecMonitor`来记录训练过程中的性能数据。

### 示例代码

```python
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.callbacks import CheckpointCallback

# 确保输出目录存在
log_dir = "./logs/"
os.makedirs(log_dir, exist_ok=True)

# 创建并包装环境
env = gym.make('LunarLander-v2')
vec_env = DummyVecEnv([lambda: env])  # 向量化环境
monitored_env = VecMonitor(vec_env, filename=os.path.join(log_dir, "monitor.csv"))  # 监控环境

# 创建PPO模型
model = PPO("MlpPolicy", monitored_env, verbose=1)

# 定义一个检查点回调，以便定期保存模型
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path=log_dir,
                                         name_prefix='rl_model')

# 训练模型
model.learn(total_timesteps=25000, callback=checkpoint_callback)

# 关闭环境
monitored_env.close()
```

### 代码解释

1. **环境准备**：
    - 使用`gym.make`创建`LunarLander-v2`环境实例。
    - 使用`DummyVecEnv`对环境进行向量化，以适配Stable Baselines3的API。
    - 使用`VecMonitor`包装器对向量化的环境进行包装，并指定输出日志文件的路径。这样，关于每个episode的奖励、长度、时间等信息将被记录。

2. **模型创建**：
    - 创建一个使用多层感知器（MLP）策略的PPO模型，并将监控过的环境传递给模型。这使得模型能够在训练过程中收集和利用环境的性能数据。

3. **训练模型**：
    - 在模型训练过程中，使用`CheckpointCallback`来定期保存模型。`save_freq=1000`表示每训练1000个时间步就保存一次模型。
    - `model.learn`函数开始模型的训练过程，`total_timesteps=25000`指定训练的总时间步数。

4. **日志记录**：
    - `VecMonitor`自动记录每个episode的性能数据到指定的CSV文件中。这些数据包括每个episode的总奖励、持续时间等，对于评估模型性能和训练过程中的进展非常有用。

5. **环境关闭**：
    - 训练完成后，通过调用`monitored_env.close()`关闭环境，释放资源。

通过这个示例，你可以看到如何结合使用`VecMonitor`和PPO算法来训练一个强化学习模型，并记录关键的训练过程数据。这些记录的数据对于分析和评估模型的学习效果非常重要，有助于理解模型在环境中的表现以及可能需要进行的调整。

## VecExtractDictObs

`VecExtractDictObs`是Stable Baselines3库中的一个向量化环境包装器，专门用于从字典类型的观察中提取特定键（key）对应的观察值。这在处理复杂环境时非常有用，特别是当环境观察返回的是包含多个不同数据的字典时。通过这个包装器，可以专注于字典中的特定部分，简化模型的输入处理。

### 参数

- `venv (VecEnv)`: 要包装的向量化环境。
- `key (str)`: 字典观察中要提取的键名。

### 方法

#### `reset()`

- 重置所有环境，并返回重置后的观察值数组。这些观察值将仅包含由`key`指定的字典部分。
- **返回类型**: `ndarray`

#### `step_wait()`

- 等待`step_async()`执行的动作完成，并返回观察值、奖励、完成标志和额外信息。返回的观察值将仅包含由`key`指定的字典部分。
- **返回类型**: `Tuple[ndarray | Dict[str, ndarray] | Tuple[ndarray, …], ndarray, ndarray, List[Dict]]`

### 使用示例

假设我们有一个环境，其观察值是一个字典，包含多个键，如`{"image": image_data, "velocity": velocity_data}`。如果我们的模型只关心图像数据，可以使用`VecExtractDictObs`来仅提取`"image"`键对应的观察值。

```python
import gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecExtractDictObs

# 创建一个返回字典观察的环境实例（这里假设已经定义了这样的环境）
class CustomEnv(gym.Env):
    # 环境的初始化、step、reset等方法实现
    ...

# 向量化环境
vec_env = DummyVecEnv([lambda: CustomEnv()])

# 使用VecExtractDictObs包装器，指定提取"image"键的观察值
extracted_env = VecExtractDictObs(vec_env, key="image")

# 重置环境，观察返回的数据
obs = extracted_env.reset()
print("Observation shape after extraction:", obs.shape)

# 执行一步动作，观察返回的数据
action = [extracted_env.action_space.sample()]
obs, rewards, dones, infos = extracted_env.step(action)
print("Observation shape after step:", obs.shape)
```

在这个示例中，`VecExtractDictObs`包装器被用来从复杂的字典观察中提取出`"image"`键对应的数据，使得向量化环境的观察值仅包含这部分数据。这样做可以简化模型的输入处理，使得模型能够专注于处理特定的观察数据。

为了提供一个完整的示例使用`VecExtractDictObs`类，我们将首先创建一个假设的环境，该环境的观察值是一个字典，包含图像数据和其他信息。然后，我们会展示如何使用`VecExtractDictObs`来仅提取图像数据用于模型训练。

### 第一步：定义环境

我们先定义一个简单的环境，它的观察空间是一个字典，包含`'image'`和`'info'`两个键。为了简化，我们假设图像数据是一个形状为`(84, 84, 3)`的数组，`'info'`是一个包含额外信息的一维数组。

```python
import gym
import numpy as np
from gym.spaces import Box, Dict

class CustomDictEnv(gym.Env):
    """一个返回字典观察值的自定义环境。"""
    def __init__(self):
        super(CustomDictEnv, self).__init__()
        self.observation_space = Dict({
            'image': Box(low=0, high=255, shape=(84, 84, 3), dtype=np.uint8),
            'info': Box(low=0, high=1, shape=(10,), dtype=np.float32)
        })
        self.action_space = Box(low=-1.0, high=1.0, shape=(3,), dtype=np.float32)

    def reset(self):
        obs = {
            'image': np.random.randint(0, 255, (84, 84, 3), dtype=np.uint8),
            'info': np.random.random((10,)).astype(np.float32)
        }
        return obs

    def step(self, action):
        obs = self.reset()  # 简化逻辑，每步都返回新的随机观察值
        reward = np.random.rand()
        done = np.random.rand() > 0.95  # 偶尔结束
        info = {}
        return obs, reward, done, info
```

### 第二步：应用VecExtractDictObs

接下来，我们将环境向量化，并使用`VecExtractDictObs`来提取字典观察中的`'image'`键对应的数据。

```python
from stable_baselines3.common.vec_env import DummyVecEnv, VecExtractDictObs

# 向量化自定义环境
vec_env = DummyVecEnv([lambda: CustomDictEnv()])

# 应用VecExtractDictObs包装器提取'image'键的观察值
extracted_env = VecExtractDictObs(vec_env, key='image')

# 重置环境和执行一些步骤
obs = extracted_env.reset()
print("Observation shape after extraction:", obs.shape)

for _ in range(5):
    action = [extracted_env.action_space.sample()]
    obs, rewards, dones, infos = extracted_env.step(action)
    print("Observation shape in step:", obs.shape)
```

### 代码解释

- **自定义环境**：我们定义了一个`CustomDictEnv`环境，它的观察空间是一个包含图像和额外信息的字典。


- **向量化和提取**：通过`DummyVecEnv`将环境向量化，使其能够与Stable Baselines3库一起使用。然后，我们利用`VecExtractDictObs`包装器仅提取字典中的`'image'`键对应的观察部分。这对于强化学习模型来说非常有用，尤其是当模型仅需要图像数据进行学习时。

- **观察值和动作**：在环境重置和每一步动作执行时，`VecExtractDictObs`确保返回的观察值仅包含我们感兴趣的图像数据。通过打印观察值的形状，我们可以看到它现在只包含图像数据，形状为`(84, 84, 3)`，这符合我们的预期。

这个示例展示了如何在复杂的观察空间中提取特定的观察部分，以适应模型的输入需求。`VecExtractDictObs`类使得处理具有字典观察空间的环境变得简单，让研究人员和开发者能够专注于模型的开发，而不必担心数据预处理的复杂性。